In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=3325d7076039d0d91806f63bda851c23625067dd3109081bfb28915a9d338975
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [64]:
from pyspark.sql import SparkSession

In [90]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [91]:
from pyspark.ml.regression import LinearRegression

In [93]:
training = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [94]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [95]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [96]:
lrModel = lr.fit(training)

In [97]:
print("Coefficients: {}".format(str(lrModel.coefficients))) # For each feature...
print('\n')
print("Intercept:{}".format(str(lrModel.intercept)))

Coefficients: [0.0073350710225801715,0.8313757584337543,-0.8095307954684084,2.441191686884721,0.5191713795290002,1.1534591903547016,-0.2989124112808717,-0.5128514186201779,-0.619712827067017,0.695615180432293]


Intercept:0.14228558260358093


In [98]:
trainingSummary = lrModel.summary

In [99]:
trainingSummary.residuals.show()
print("RMSE: {}".format(trainingSummary.rootMeanSquaredError))
print("r2: {}".format(trainingSummary.r2))

+-------------------+
|          residuals|
+-------------------+
|-11.011130022096554|
| 0.9236590911176535|
|-4.5957401897776675|
|  -20.4201774575836|
|-10.339160314788181|
|-5.9552091439610555|
|-10.726906349283922|
| 2.1228071931912336|
|  4.077122222293811|
|-17.316168071241652|
| -4.593044343959059|
|  6.380476690746936|
| 11.320566035059846|
|-20.721971774534094|
| -2.736692773777401|
| -16.66886934252847|
|  8.242186378876315|
|-1.3723486332690233|
|-0.7060332131264666|
|-1.1591135969994064|
+-------------------+
only showing top 20 rows

RMSE: 10.16309157133015
r2: 0.027839179518600154


In [100]:
all_data = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [101]:
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [102]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|
|-17.026492264209548|(10,[0,1,2,3,4,5,...|
|-16.692207021311106|(10,[0,1,2,3,4,5,...|
| -16.26143027545273|(10,[0,1,2,3,4,5,...|
+----------

In [103]:
test_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
|-16.151349351277112|(10,[0,1,2,3,4,5,...|
| -15.72351561304857|(10,[0,1,2,3,4,5,...|
|-15.437384793431217|(10,[0,1,2,3,4,5,...|
|-15.310980589416289|(10,[0,1,2,3,4,5,...|
|-14.762758252931127|(10,[0,1,2,3,4,5,...|
|-14.328978509075442|(10,[0,1,2,3,4,5,...|
|-13.867087895158768|(10,[0,1,2,3,4,5,...|
|-12.977848725392104|(10,[0,1,2,3,4,5,...|
|-12.198096564661412|(10,[0,1,2,3,4,5,...|
|-12.094351278535258|(10,[0,1,2,3,4,5,...|
| -11.87816749996684|(10,[0,1,2,3,4,5,...|
|-11.857350365429426|(10,[0,1,2,3,4,5,...|
+----------

In [104]:
unlabeled_data = test_data.select('features')

In [105]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [106]:
correct_model = lr.fit(train_data)

In [107]:
test_results = correct_model.evaluate(test_data)

In [108]:
test_results.residuals.show()
print("RMSE: {}".format(test_results.rootMeanSquaredError))

+-------------------+
|          residuals|
+-------------------+
|-27.157310730503998|
|-23.940250441359805|
| -26.68342388217344|
| -21.80224530732658|
|-21.827890738206342|
|-19.956461618995984|
| -16.04332242011562|
|-17.113562197247123|
|-17.254888492702328|
|  -18.5676328041694|
|-15.201744337120658|
|-14.023219694247357|
|-14.415997528079586|
|-17.779334734604916|
|-11.251288725353218|
|-13.166504213425439|
|-16.165897316381347|
|-11.705027490908048|
|-11.369198277880647|
|-10.012300309867989|
+-------------------+
only showing top 20 rows

RMSE: 10.2618409233819


In [109]:
predictions = correct_model.transform(unlabeled_data)

In [110]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -0.8887073072726367|
|(10,[0,1,2,3,4,5,...|  -2.795956741241919|
|(10,[0,1,2,3,4,5,...|  3.7335979459773667|
|(10,[0,1,2,3,4,5,...|   1.590168048367909|
|(10,[0,1,2,3,4,5,...|   1.943329963932918|
|(10,[0,1,2,3,4,5,...|   1.110539146097402|
|(10,[0,1,2,3,4,5,...|  -1.450877936767725|
|(10,[0,1,2,3,4,5,...|  0.3944653636420346|
|(10,[0,1,2,3,4,5,...|  1.1035391414252147|
|(10,[0,1,2,3,4,5,...|   2.844117191120829|
|(10,[0,1,2,3,4,5,...|-0.23564045631055833|
|(10,[0,1,2,3,4,5,...|  -1.287760895168931|
|(10,[0,1,2,3,4,5,...|-0.34676072485153997|
|(10,[0,1,2,3,4,5,...|  3.4503562255294717|
|(10,[0,1,2,3,4,5,...| -2.6157991698055496|
|(10,[0,1,2,3,4,5,...|  0.1886554880333348|
|(10,[0,1,2,3,4,5,...|  3.9678007517199343|
|(10,[0,1,2,3,4,5,...|-0.38932378762720976|
|(10,[0,1,2,3,4,5,...| -0.5089692220861931|
|(10,[0,1,2,3,4,5,...|  -1.84505